## Incremental MSBWT Construction

In [1]:
def FMIndex(bwt):
    fm = [{c: 0 for c in bwt}]
    for c in bwt:
        row = {symbol: count + 1 if (symbol == c) else count for symbol, count in fm[-1].items()}
        fm.append(row)
    offset = {}
    N = 0
    for symbol in sorted(row.keys()):
        offset[symbol] = N
        N += row[symbol]
    return fm, offset

def inverseBWT(bwt):
    # initialize the table from t
    table = ['' for c in bwt]
    for j in range(len(bwt)):
        #insert the BWT as the first column
        table = sorted([c+table[i] for i, c in enumerate(bwt)])
    #return the row that ends with ‘$’
    return table[bwt.index('$')]

def BWT(t):
    # create a sorted list of all cyclic suffixes of t
    rotation = sorted([t[i:]+t[:i] for i in range(len(t))])
    # concatenate the last symbols from each suffix
    return ''.join(r[-1] for r in rotation)

def recoverSuffix(i, BWT, FMIndex, Offset):
    suffix = ''
    c = BWT[i]
    predec = Offset[c] + FMIndex[i][c]
    suffix = c + suffix
    while (predec != i):
        c = BWT[predec]
        predec = Offset[c] + FMIndex[predec][c]
        suffix = c + suffix
    return suffix

def findBWT(pattern, FMIndex, Offset):
    lo = 0
    hi = len(FMIndex) - 1
    for symbol in reversed(pattern):
        lo = Offset[symbol] + FMIndex[lo][symbol]
        hi = Offset[symbol] + FMIndex[hi][symbol]
    return lo, hi

In [26]:
bwt1 = BWT('TAGACAT$')
bwt2 = BWT('GAGATAG$')

print(bwt1, bwt2)

bwt = bwt1

fm, off = FMIndex(bwt)
for i in range(len(bwt)):
    print("%2d: %s %s %s" % (i, recoverSuffix(i, bwt, fm, off), bwt[i], [fm[i][c] for c in "$ACGT"]))
print("               %s" % [fm[i+1][c] for c in "$ACGT"])
print()


print(findBWT("CAT", fm, off))

bwt = bwt2

fm, off = FMIndex(bwt)
for i in range(len(bwt)):
    print("%2d: %s" % (i, recoverSuffix(i, bwt, fm, off)))
print()
print(findBWT("TAG", fm, off))

TGTCAAA$ GTGGA$AA
 0: $TAGACAT T [0, 0, 0, 0, 0]
 1: ACAT$TAG G [0, 0, 0, 0, 1]
 2: AGACAT$T T [0, 0, 0, 1, 1]
 3: AT$TAGAC C [0, 0, 0, 1, 2]
 4: CAT$TAGA A [0, 0, 1, 1, 2]
 5: GACAT$TA A [0, 1, 1, 1, 2]
 6: T$TAGACA A [0, 2, 1, 1, 2]
 7: TAGACAT$ $ [0, 3, 1, 1, 2]
               [1, 3, 1, 1, 2]

(4, 5)
 0: $GAGATAG
 1: AG$GAGAT
 2: AGATAG$G
 3: ATAG$GAG
 4: G$GAGATA
 5: GAGATAG$
 6: GATAG$GA
 7: TAG$GAGA

(7, 8)


In [ ]:
TGTCAAA$ GTGGA$AA

T,0 $,0 T,0 $,1 
G,0 $,1 G,1 $,0
T,0 A,0 G,0 A,0
C,0 A,0 T,0 A,0
A,0 A,0 C,0 A,1
A,0 A,1 T,1 A,1
A,0 A,1 G,1 A,0
$,0 A,1 G,1 A,1
G,1 C,0 A,0 C,0
T,1 G,0 A,0 G,1
G,1 G,1 A,1 G,0
G,1 G,1 $,1 G,1
A,1 G,1 A,1 G,1
$,1 T,0 A,0 T,0
A,1 T,0 $,0 T,0
A,1 T,1 A,1 T,1

In [ ]:
# step 1: sort based on letters
# step 2: select from bwt based on numbers
# step 3: sort based on letters
# step 4: keep repeating until numbers don't change when sort based on letters

In [2]:
# Constructing a multistring BWT one suffix at a time
# first let's take a look at the implicit suffix array
bwt = "ATTA$T$CT$AAA$"
fm, off = FMIndex(bwt)
for i in range(len(bwt)):
    print("%2d: %s" % (i, recoverSuffix(i, bwt, fm, off)))
print()

# New string to include
new = "TATA$"
inserts = []
for i in range(len(new)):
    rotation = new[i:]+new[:i]
    l, h = findBWT(rotation,fm,off)
    inserts.append((h,new[i-1],rotation))
print(inserts)

# Insert into original BWT in reverse order
for i, c, rot in sorted(inserts, reverse=True):
    bwt = bwt[:i] + c + bwt[i:]

# Look at result
print(bwt)
print()
fm, off = FMIndex(bwt)
for i in range(len(bwt)):
    print("%2d: %s" % (i, recoverSuffix(i, bwt, fm, off)))

 0: $A
 1: $AT
 2: $CAT
 3: $TATA
 4: A$
 5: A$TAT
 6: AT$
 7: AT$C
 8: ATA$T
 9: CAT$
10: T$A
11: T$CA
12: TA$TA
13: TATA$

[(14, '$', 'TATA$'), (9, 'T', 'ATA$T'), (13, 'A', 'TA$TA'), (6, 'T', 'A$TAT'), (4, 'A', '$TATA')]
ATTAA$TT$CTT$AAAA$$

 0: $A
 1: $AT
 2: $CAT
 3: $TATA
 4: $TATA
 5: A$
 6: A$TAT
 7: A$TAT
 8: AT$
 9: AT$C
10: ATA$T
11: ATA$T
12: CAT$
13: T$A
14: T$CA
15: TA$TA
16: TA$TA
17: TATA$
18: TATA$


In [3]:
# Constructing a multistring BWT one suffix at a time
# first let's take a look at the implicit suffix array
bwt = "TGAG$TGC$AAA$AA"
fm, off = FMIndex(bwt)
for i in range(len(bwt)):
    print("%2d: %s" % (i, recoverSuffix(i, bwt, fm, off)))
print()

# New string to include
new = "TATA$"
inserts = []
for i in range(len(new)):
    rotation = new[i:]+new[:i]
    l, h = findBWT(rotation,fm,off)
    inserts.append((h,new[i-1],rotation))
print(inserts)

# Insert into original BWT in reverse order
for i, c, rot in sorted(inserts, reverse=True):
    bwt = bwt[:i] + c + bwt[i:]

# Look at result
print(bwt)
print()
fm, off = FMIndex(bwt)
for i in range(len(bwt)):
    print("%2d: %s" % (i, recoverSuffix(i, bwt, fm, off)))

 0: $ACAT
 1: $ATAG
 2: $GAGA
 3: A$GAG
 4: ACAT$
 5: AG$AT
 6: AGA$G
 7: AT$AC
 8: ATAG$
 9: CAT$A
10: G$ATA
11: GA$GA
12: GAGA$
13: T$ACA
14: TAG$A

[(15, '$', 'TATA$'), (8, 'T', 'ATA$T'), (14, 'A', 'TA$TA'), (4, 'T', 'A$TAT'), (3, 'A', '$TATA')]
TGAAGT$TGCT$AAA$AAA$

 0: $ACAT
 1: $ATAG
 2: $GAGA
 3: $TATA
 4: A$GAG
 5: A$TAT
 6: ACAT$
 7: AG$AT
 8: AGA$G
 9: AT$AC
10: ATA$T
11: ATAG$
12: CAT$A
13: G$ATA
14: GA$GA
15: GAGA$
16: T$ACA
17: TA$TA
18: TAG$A
19: TATA$


In [4]:
# Try it
fm, off = FMIndex(bwt)
l, h = findBWT("ATA", fm, off)
print(l,h)
for i in range(l,h):
    print(recoverSuffix(i,bwt,fm,off))

10 12
ATA$T
ATAG$


## msBWT merging in Python

In [10]:
def mergeBWT(bwt1, bwt2):
    interleave = [(c, 0) for c in bwt1] + [(c, 1) for c in bwt2]
    passes = min(len(bwt1), len(bwt2))
    for p in range(passes):
        i, j = 0, 0
        nextInterleave = []
        for c, k in sorted(interleave, key=lambda x: x[0]):
            if (k == 0):
                b = bwt1[i]
                i += 1
            else:
                b = bwt2[j]
                j += 1
            nextInterleave.append((b, k))
        if (nextInterleave == interleave):
            break
        interleave = nextInterleave
    return ''.join([c for c, k in interleave])

bwt1 = "TG$TC$AAAA"
bwt2 = "AAGTGTA$A$"
bwt12 = mergeBWT(bwt1, bwt2)

print(bwt12)
FM, Offset = FMIndex(bwt12)
for i in range(len(bwt12)):
    j = (i>>2)+(i&3)*(len(bwt12)//4)
    print("%2d: %s" % (j, recoverSuffix(j,bwt12,FM,Offset)), "\n" if (i % 4 == 3) else "", end='')

TGAAGT$TGCT$AAA$AAA$
 0: $ACAT  5: A$TAT 10: ATA$T 15: GAGA$ 
 1: $ATAG  6: ACAT$ 11: ATAG$ 16: T$ACA 
 2: $GAGA  7: AG$AT 12: CAT$A 17: TA$TA 
 3: $TATA  8: AGA$G 13: G$ATA 18: TAG$A 
 4: A$GAG  9: AT$AC 14: GA$GA 19: TATA$ 


In [12]:
bwt1 = BWT("ATACATTAG$")
bwt2 = BWT("CATCAT$")
print(bwt1,bwt2)
bwt = mergeBWT(bwt1,bwt2)
print(bwt)
fm, off = FMIndex(bwt)
for i in range(len(bwt)):
    print("%2d: %s" % (i, recoverSuffix(i, bwt, fm, off)))

GTT$CAAATA TCCT$AA
GTTTC$CCT$AAAATAA
 0: $ATACATTAG
 1: $CATCAT
 2: ACATTAG$AT
 3: AG$ATACATT
 4: AT$CATC
 5: ATACATTAG$
 6: ATCAT$C
 7: ATTAG$ATAC
 8: CAT$CAT
 9: CATCAT$
10: CATTAG$ATA
11: G$ATACATTA
12: T$CATCA
13: TACATTAG$A
14: TAG$ATACAT
15: TCAT$CA
16: TTAG$ATACA
